In [2]:
# import os, pyodbc, time, unidecode
from utils.df_handle import *

In [3]:
# day_ago = 2
datenow = datetime.now().strftime("%Y%m%d")
# datenow_day_ago = ( datetime.now()-timedelta(day_ago) ).strftime("%Y%m%d")
# param_1 = f"'{datenow_day_ago}'"
param_2 = f"'20210901'"
param_3 = f"'{datenow}'"
param_4 = f"'20211109'"


In [4]:
query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_v1] @Fromdate={param_3}, @Todate={param_3}"

In [5]:
FINAL = get_ms_df(sql=query)

In [16]:
if FINAL.shape[0] != 0:
    print('do it')
else:
    print('not do it')


True

In [30]:
# df1['Nhãn Hàng'].to_clipboard()

In [31]:
# FINAL.columns = cleancols(FINAL)

In [32]:
# FINAL.columns

In [33]:
# get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'")

In [12]:
# FINAL.NgayChungTu.shape[0]

0

In [ ]:
# query = f"EXEC [pr_OM_RawdataSellOutPayroll_BI_v1] @Fromdate={param_1}, @Todate={param_2}"
# get_ms_csv(query, 'sales_2.csv')
# FINAL1 = pd.read_csv('sales_2.csv')
# FINAL = pd.read_csv('sales.csv')
FINAL.columns = cleancols(FINAL)
# FINAL.columns
# FINAL.DoanhSoChuaVAT.sum()
FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

FINAL['phanloaispcl'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, phanloai from d_nhom_sp where nhomsp='SPCL'"))
).fillna('Khác')

FINAL['nhomsp'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp IN ('SPCL', 'SP MOI') "))
).fillna('Khác')

FINAL['khuvucviettat'] = FINAL['TenKhuVuc'].map(
    df_to_dict(get_ps_df("select * from d_mkv_viet_tat"))
)

FINAL['chinhanh'] = FINAL['MaCongTyCN'].map(
    df_to_dict(get_ps_df("select * from d_chi_nhanh"))
)

FINAL['newhco'] = (FINAL['MaKenhPhu']+FINAL['MaPhanLoaiHCO']).map(
    df_to_dict(get_ps_df("SELECT concat(makenhphu, maphanloaihco) as concat, new_mahco FROM d_pl_hco"))
)

FINAL['phanam'] = FINAL['MaSanPham'].map(
    df_to_dict(get_ps_df("select masanpham, nhomsp from d_nhom_sp where nhomsp='PHA NAM'"))).fillna('Merap')

FINAL['thang'] = FINAL['NgayChungTu'] + pd.offsets.Day() - pd.offsets.MonthBegin()

FINAL['inserted_at'] = datetime.now()

In [35]:
# FINAL.NgayGiaoHang.fillna(datetime(1900, 1, 1), inplace=True)

In [36]:
# vc(FINAL, 'NgayGiaoHang').to_clipboard()

In [37]:
# FINAL.dtypes

In [38]:
# FINAL.columns = lower_col(FINAL)

In [39]:
FINAL.columns

Index(['MaCongTyCN', 'CongTyCN', 'NgayChungTu', 'SoDonDatHang', 'SoDonTraHang',
       'NgayTraHang', 'HoaDon', 'TrangThai', 'MaKHThue', 'MaKHCu',
       'TenKhachHang', 'TenVungBH', 'TenKhuVuc', 'TenTinhKH', 'MaKenhKH',
       'TenKenhKH', 'MaKenhPhu', 'TenKenhPhu', 'MaHCO', 'TenHCO',
       'MaPhanLoaiHCO', 'TenPhanLoaiHCO', 'MaPhanHangHCO', 'TenPhanHangHCO',
       'NhanHang', 'MaSanPham', 'TenSanPhamNB', 'TenSanPhamVietTat', 'SoLo',
       'LineRef', 'SoLuong', 'DonGiaCoVAT', 'DoanhSoCoVAT', 'DonGiaChuaVAT',
       'DoanhSoChuaVAT', 'NgayDatDon', 'NgayGiaoHang', 'MaNV', 'TenCVBH',
       'TenQuanLyTT', 'TenQuanLyKhuVuc', 'TenQuanLyVung', 'MaNVGH',
       'NguoiGiaohang', 'TrangThaiGiaoHang', 'DonViGiaoHang',
       'TenNhaVanChuyen', 'KieuDonHang', 'phanloaispcl', 'nhomsp',
       'khuvucviettat', 'chinhanh', 'newhco', 'phanam', 'thang',
       'inserted_at'],
      dtype='object')

In [40]:
pk = ['macongtycn', 'ngaychungtu', 'sodondathang', 'masanpham', 'solo', 'lineref', 'soluong']

In [41]:
# df_filter(FINAL, sodondathang='OO102021-00001').to_clipboard()

In [42]:
execute_values_upsert(FINAL, 'f_sales', pk=pk)

INSERT INTO f_sales(MaCongTyCN,CongTyCN,NgayChungTu,SoDonDatHang,SoDonTraHang,NgayTraHang,HoaDon,TrangThai,MaKHThue,MaKHCu,TenKhachHang,TenVungBH,TenKhuVuc,TenTinhKH,MaKenhKH,TenKenhKH,MaKenhPhu,TenKenhPhu,MaHCO,TenHCO,MaPhanLoaiHCO,TenPhanLoaiHCO,MaPhanHangHCO,TenPhanHangHCO,NhanHang,MaSanPham,TenSanPhamNB,TenSanPhamVietTat,SoLo,LineRef,SoLuong,DonGiaCoVAT,DoanhSoCoVAT,DonGiaChuaVAT,DoanhSoChuaVAT,NgayDatDon,NgayGiaoHang,MaNV,TenCVBH,TenQuanLyTT,TenQuanLyKhuVuc,TenQuanLyVung,MaNVGH,NguoiGiaohang,TrangThaiGiaoHang,DonViGiaoHang,TenNhaVanChuyen,KieuDonHang,phanloaispcl,nhomsp,khuvucviettat,chinhanh,newhco,phanam,thang,inserted_at) VALUES %s ON CONFLICT (macongtycn, ngaychungtu, sodondathang, masanpham, solo, lineref, soluong) DO UPDATE SET MaCongTyCN = excluded.MaCongTyCN, CongTyCN = excluded.CongTyCN, NgayChungTu = excluded.NgayChungTu, SoDonDatHang = excluded.SoDonDatHang, SoDonTraHang = excluded.SoDonTraHang, NgayTraHang = excluded.NgayTraHang, HoaDon = excluded.HoaDon, TrangThai = e

In [39]:
# FINAL2 = FINAL.head()

In [40]:
# FINAL2.to_csv('FINAL.csv')

In [24]:
############# ENDDDDDDDDDDD #################